In [36]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import model_selection,preprocessing,metrics
import sys
sys.path.append("../Recursos")
from ClassNeuronaLineal import NeuronaLineal


# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv("../Datos/automobile-simple.csv")

df

,make,fuel-type,num-of-doors,body-style,curb-weight,engine-size,horsepower,city-mpg,highway-mpg,price,volume,eco-rating
0,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,13495.0,528019.904,33.297462
1,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,16500.0,528019.904,33.297462
2,alfa-romero,gas,two,hatchback,2823,152,154.0,19,26,16500.0,587592.640,30.898272
3,audi,gas,four,sedan,2337,109,102.0,24,30,13950.0,634816.956,42.697819
4,audi,gas,four,sedan,2824,136,115.0,18,22,17450.0,636734.832,27.997459
...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,four,sedan,2952,141,114.0,23,28,16845.0,721961.760,38.354298
201,volvo,gas,four,sedan,3049,141,160.0,19,25,19045.0,720913.920,32.043335
202,volvo,gas,four,sedan,3012,173,134.0,18,23,21485.0,721961.760,29.806494
203,volvo,diesel,four,sedan,3217,145,106.0,26,27,22470.0,721961.760,35.073342


In [37]:
#-- seleccionar los atributos numéricos --
df = df.select_dtypes(include = ["int16", "int32", "int64", "float16", "float32", "float64"])

# imprime valores nulos
print(df.isnull().sum())

nomAtrib = np.array(df.columns.values).reshape(-1,1)

#-- ver si tienen datos faltantes --
values = {'curb-weight	': df['curb-weight'].mean(),
          'engine-size': df['engine-size'].mean(),
          'horsepower': df['horsepower'].mean(),
          'city-mpg': df['city-mpg'].mean(),
          'highway-mpg': df['highway-mpg'].mean(),
          'price': df['price'].mean(),
          'volume': df['volume'].mean(),
          'eco-rating': df['eco-rating'].mean()
          }

# reemplaza valores nulos por los indicados en el diccionario values
df = df.fillna(value=values)

# Imprime matriz de correclacion
corr = df.corr()

corr

curb-weight    0
engine-size    0
horsepower     2
city-mpg       0
highway-mpg    0
price          4
volume         0
eco-rating     0
dtype: int64


,curb-weight,engine-size,horsepower,city-mpg,highway-mpg,price,volume,eco-rating
curb-weight,1.000000,0.850594,0.750968,-0.757414,-0.797465,0.820825,0.819806,-0.842111
engine-size,0.850594,1.000000,0.810713,-0.653658,-0.677470,0.861752,0.594351,-0.719602
horsepower,0.750968,0.810713,1.000000,-0.803162,-0.770903,0.757917,0.429366,-0.816144
city-mpg,-0.757414,-0.653658,-0.803162,1.000000,0.971337,-0.667449,-0.550292,0.977349
highway-mpg,-0.797465,-0.677470,-0.770903,0.971337,1.000000,-0.690526,-0.602410,0.983781
price,0.820825,0.861752,0.757917,-0.667449,-0.690526,1.000000,0.623298,-0.725885
volume,0.819806,0.594351,0.429366,-0.550292,-0.602410,0.623298,1.000000,-0.593856
eco-rating,-0.842111,-0.719602,-0.816144,0.977349,0.983781,-0.725885,-0.593856,1.000000


In [38]:
Y = df['price'].values
normalizador_x,normalizador_y = preprocessing.StandardScaler(), preprocessing.StandardScaler()
df_filtrado = df.drop(columns=['price'])
X = df_filtrado.values
X = normalizador_x.fit_transform(X)
Y = normalizador_y.fit_transform(Y.reshape(-1,1))


In [39]:
Pesos = []
for i in range(5):
    combinador = NeuronaLineal(alpha=0.001, n_iter=100, cotaE=10e-06, draw=1, title=['X', 'Y'])
    combinador.fit(X,Y)
    Pesos.append(combinador.w_)

Pesos


[array([ 0.17716012,  0.46234379,  0.18696658,  0.22419392, -0.12646527,
         0.08528544, -0.11642855]),
 array([ 0.10192019,  0.53381939,  0.16041406,  0.07314039, -0.13281382,
         0.11763712,  0.02375895]),
 array([ 0.02557463,  0.47941924,  0.22599368,  0.37029492, -0.10090087,
         0.15825348, -0.32306248]),
 array([ 0.12086214,  0.51826795,  0.14192028,  0.13147289, -0.01567252,
         0.10337302, -0.17106157]),
 array([-0.10799242,  0.57330434,  0.18295068,  0.35165343, -0.09996929,
         0.20103615, -0.36250543])]

In [44]:
entrada = np.array([[3012,173,134.0,18,23,721961.760,29.806494],[2824,136,115.0,18,22,636734.832,27.997459],[2952,141,114.0,23,28,721961.760,38.354298]])

entrada = normalizador_x.transform(entrada)

prediccion = combinador.predict(entrada)

prediccion = normalizador_y.inverse_transform([prediccion])

prediccion
# 21485.0
# 17450.0
# 16845.0

array([[21278.62121239, 15722.18954554, 16742.69044856]])